# Evn

In [ ]:
import os
import random
import shutil
import json
import zipfile
import math
import copy
import collections
import re

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K

from tqdm.notebook import tqdm

In [ ]:
# random seed initialize
random_seed = 1234
random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

In [ ]:
!nvidia-smi

# 입력 및 Vocab

In [ ]:
# 입력 문장
sentences = [
    '나는 오늘 기분이 좋아 나는 오늘 우울해'
]

In [ ]:
# 각 문장을 띄어쓰기 단위로 분할
words = []
for sentence in sentences:
    words.extend(sentence.split())

# 중복 단어 제거
words = list(dict.fromkeys(words))

# 각 단어별 고유한 번호 부여
word_to_id = {'[PAD]': 0, '[UNK]': 1}
for word in words:
    word_to_id[word] = len(word_to_id)

# 각 숫자별 단어 부여
id_to_word = {_id:word for word, _id in word_to_id.items()}

word_to_id, id_to_word

In [ ]:
# 학습용 입력 데이터 생성
train_inputs = []
for sentence in sentences:
    train_inputs.append([word_to_id[word] for word in sentence.split()])

# train inputs을 numpy array로 변환
train_inputs = np.array(train_inputs)

train_inputs

In [ ]:
# embedding
embedding = tf.keras.layers.Embedding(len(word_to_id), 4)

In [ ]:
# 단어벡터
hidden = embedding(train_inputs)
hidden

# CNN

# padding valid

In [ ]:
# CNN (padding valid)
conv_1 = tf.keras.layers.Conv1D(filters=5, kernel_size=3, padding='valid')  # 기본 값

In [ ]:
# CNN 실행
output_1 = conv_1(hidden)  # (bs, n_seq - kernel_size + 1, filters)
output_1

In [ ]:
# weights
weights = conv_1.get_weights()
W = weights[0]
b = weights[1]
print(W)  # (kernel_size, input_hidden, filters)
print(b)  # (filters,)

# padding causal

In [ ]:
# CNN (padding causal)
conv_2 = tf.keras.layers.Conv1D(filters=5, kernel_size=3, padding='causal')

In [ ]:
# CNN 실행
output_2 = conv_2(hidden)  # (bs, n_seq, filters)
output_2.shape

In [ ]:
# weights
weights = conv_2.get_weights()
W = weights[0]
b = weights[1]
print(W)  # (kernel_size, input_hidden, filters)
print(b)  # (filters,)

# padding same

In [ ]:
# CNN (padding same)
conv_3 = tf.keras.layers.Conv1D(filters=5, kernel_size=3, padding='same')

In [ ]:
# CNN 실행
output_3 = conv_3(hidden)  # (bs, n_seq, filters)
output_3.shape

In [ ]:
# weights
weights = conv_3.get_weights()
W = weights[0]
b = weights[1]
print(W.shape)  # (kernel_size, input_hidden, filters)
print(b.shape)  # (filters,)

# dilation_rate

In [ ]:
# CNN (padding valid, dilation_rate)
conv_4 = tf.keras.layers.Conv1D(filters=5, kernel_size=3, padding='valid', dilation_rate=2)

In [ ]:
# CNN 실행
output_4 = conv_4(hidden)  # (bs, n_seq - window_size + 1, filters), window_size = kernel_size + (kernel_size - 1) * (dilation_rate - 1)
output_4.shape

In [ ]:
# weights
weights = conv_4.get_weights()
W = weights[0]
b = weights[1]
print(W.shape)  # (kernel_size, input_hidden, filters)
print(b.shape)  # (filters,)

# CNN 모델

In [ ]:
def build_model(n_vocab, d_model, n_seq, n_out):
    """
    CNN 모델
    :param n_vocab: vocabulary 단어 수
    :param d_model: 단어를 의미하는 벡터의 차원 수
    :param n_seq: 문장길이 (단어 수)
    :param n_out: 예측할 class 개수
    """
    input = tf.keras.layers.Input(shape=(n_seq,))

    embedding = tf.keras.layers.Embedding(n_vocab, d_model)  # (n_vocab, d_model)
    hidden = embedding(input)  # (bs, 1, d_model)

    #############################################
    conv_1 = tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='causal')
    hidden = conv_1(hidden)
    conv_2 = tf.keras.layers.Conv1D(filters=64, kernel_size=3)
    hidden = conv_2(hidden)
    #############################################

    output = tf.keras.layers.Dense(n_out, activation=tf.nn.softmax)(hidden)  # (bs, 1, n_vocab)

    model = tf.keras.Model(inputs=input, outputs=output)
    return model

In [ ]:
# 모델 생성
model = build_model(len(word_to_id), 8, 7, 3)
# 모델 내용 그래프 출력
tf.keras.utils.plot_model(model, 'model.png', show_shapes=True)

In [ ]:
model.predict(train_inputs)